In [1]:
import geopandas as gpd
from folium import LayerControl


from core.downloads.geosampa import get_capabilities, get_features

# Promoção da Sustentabilidade Ambiental, Gestão de risco

## Formulário 7

O formulário associado a este notebook solicita os dados sobre 
`áreas de risco (hidrológico e deslizamentos)`. A justificativa para estes dados é a seguinte:

> As áreas de risco estão relacionadas à construção de moradias, em sua maioria em condições precárias, em locais com geológico-geotécnicas frágeis, não recomendadas para ocupação. O impacto de chuvas concentradas intensas, características de eventos climáticos extremos deve ser monitorado pelo Município, através de políticas públicas de gestão de risco e promoção da resiliência climática.

In [ ]:
get_capabilities('hidrológico')

In [ ]:
get_capabilities('deslizamento')

Como o formulário cita apenas a camada `proteção e defesa civil/mapeamento de areas de risco`, assumirei que a referência a deslizamentos seja sobre a camada de risco geológico.

In [ ]:
df_hid = get_features('geoportal:risco_hidrologico')
df_hid.head()

In [ ]:
df_geo = get_features('geoportal:area_risco_geologico')
df_geo.head()

Além dos dados de riscos hidrogeológicos, também precisaremos dos dados do Censo de 2022 para a estimativa populacional. Os dados básicos, como número de domicílios e população, são disponibilizados diretamente no geopackage com as geometrias de setores censitários.

In [ ]:
df_censo = gpd.read_file('https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/setores/gpkg/UF/SP/SP_setores_CD2022.gpkg')
df_censo.head()

In [ ]:
df_censo = df_censo.loc[df_censo['CD_MUN']=='3550308']
df_censo.head()

## Ajustando as projeções

Vamos revisar os sistemas de coordenadas de todos os geodataframes para garantir que estão na mesma projeção.

In [ ]:
for gdf in [df_geo, df_hid, df_censo]:
    print(gdf.columns[:5])
    print(gdf.crs)

Como o geodataframe do censo está em outro crs, precisamos convertê-lo para o `epsg:31983`.

In [9]:
df_censo = df_censo.to_crs('EPSG:31983')

# Calculando a população de cada área de risco

Primeiro, vamos inspecionar visualmente os geodataframes.

In [ ]:
df_geo.explore()

In [ ]:
df_censo.iloc[:500].explore()

Depois, precisaremos calcular a interseção de cada um dos dataframes de risco com os setores censitários.

In [ ]:
ol1 = gpd.overlay(df_geo, df_censo,
            how='intersection',
            keep_geom_type=True)
ol1.head()

Vamos avaliar visualmente o resultado da interseção com base na primeira área de risco.

In [ ]:
id_area = ol1.loc[:, 'id'].iloc[0]
id_area

In [ ]:
cd_setor_list = ol1.loc[ol1['id']==id_area, 'CD_SETOR'].tolist()
ol1.loc[ol1['id']=='area_risco_geologico.1', ['id', 'CD_SETOR']]

In [ ]:
m = df_censo[df_censo['CD_SETOR'].isin(
    cd_setor_list)].explore(name='setor censitário')

m = df_geo[df_geo['id'] == 'area_risco_geologico.1'].explore(
    m=m, color='orange', name='area de risco')

filtered_ol1 = ol1.loc[ol1['id'] == 'area_risco_geologico.1', [
    'id', 'CD_SETOR', 'geometry']]
m = filtered_ol1.explore(m=m, color='purple', name='interseção')

LayerControl().add_to(m)

m

Na inspeção visual, nota-se que algumas interseções não aparentam representar áreas com moradias, mas simplesmente leves discrepâncias no desenho dos polígonos sobre áreas não populadas (ruas,  canteiros, etc.).

Por isso, precisaremos limpar essas interseções da nossa base. Podemos utilizar um buffer negativo nos polígonos, de modo que polígonos com altura/largura menor do que a metade do valor aplicado no buffer se tornarão vazios.

In [ ]:
# Assumindo que um terreno com 8m ou menos de largura não possui uma casa
buffer = -1*(8/2)

ol2 = ol1.copy()
ol2['debuffed'] = ol2.buffer(buffer)
ol2 = ol2[~ol2['debuffed'].is_empty]
ol2.head()

In [ ]:
m = df_censo[df_censo['CD_SETOR'].isin(
    cd_setor_list)].explore(name='setor censitário')

m = df_geo[df_geo['id'] == 'area_risco_geologico.1'].explore(
    m=m, color='orange', name='area de risco')

filtered_ol2 = ol2.loc[ol2['id'] == 'area_risco_geologico.1', [
    'id', 'CD_SETOR', 'geometry']]
m = filtered_ol2.explore(m=m, color='purple', name='interseção')

LayerControl().add_to(m)

m